In [1]:
%%time
import csv
import pandas as pd
import numpy as np

## Load data
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

# train
right = pd.read_csv('graph_features_scaled_train.csv')
del right['Unnamed: 0']
left = pd.read_csv('training_features_6.csv', header = None)
train = pd.concat([left, right], axis=1)

X_train = train.as_matrix()

y_train = [i[2] for i in training_set ]

# test
right = pd.read_csv('graph_features_scaled_test.csv')
del right['Unnamed: 0']
left = pd.read_csv('testing_features_6.csv', header = None)
test = pd.concat([left, right], axis=1)

X_test = test.as_matrix()



CPU times: user 2.3 s, sys: 152 ms, total: 2.45 s
Wall time: 2.45 s


In [ ]:
%%time

from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
# initialize basic SVM
clf = ExtraTreesClassifier(max_features=None, min_samples_leaf= 20, n_estimators = 500, n_jobs= 3)

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
%%time
from sklearn import cross_validation

from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
clf = ExtraTreesClassifier(max_features=None, min_samples_leaf= 10, n_estimators = 500, n_jobs= 3)
cv = cross_validation.cross_val_score(clf, X_train, y_train, cv=5)

In [8]:
%%time 
import xgboost as xgb
from sklearn import cross_validation
gbm = xgb.XGBClassifier(max_depth=6, n_estimators=500, learning_rate=0.01)
cv = cross_validation.cross_val_score(gbm, X_train, y_train, cv=5)
print np.mean(cv)

0.967451487481
CPU times: user 39min 28s, sys: 17.2 s, total: 39min 45s
Wall time: 10min 48s


In [5]:
%%time
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05)
gbm.fit(X_train, y_train)
pred = gbm.predict(X_test)

CPU times: user 6min 4s, sys: 1.94 s, total: 6min 6s
Wall time: 1min 33s


In [2]:
%%time
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
parameters = {'n_estimators':[500,1000],
        'learning_rate': [0.05, 0.01, 0.001]}

clf = GridSearchCV( xgb.XGBClassifier(max_depth=4), parameters, n_jobs=4, cv=5, verbose = 10)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 12.3min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 24.9min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 43.3min
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed: 69.9min finished


CPU times: user 7min 4s, sys: 4.3 s, total: 7min 8s
Wall time: 1h 11min 30s
[CV] n_estimators=500, learning_rate=0.05 ............................
[CV] n_estimators=500, learning_rate=0.05 ............................
[CV] n_estimators=500, learning_rate=0.05 ............................
[CV] n_estimators=500, learning_rate=0.05 ............................
[CV] ... n_estimators=500, learning_rate=0.05, score=0.967686 - 6.1min[CV] ... n_estimators=500, learning_rate=0.05, score=0.968043 - 6.0min[CV] ... n_estimators=500, learning_rate=0.05, score=0.967393 - 6.1min[CV] ... n_estimators=500, learning_rate=0.05, score=0.967677 - 6.0min



[CV] n_estimators=500, learning_rate=0.05 ............................[CV] n_estimators=1000, learning_rate=0.05 ...........................[CV] n_estimators=1000, learning_rate=0.05 ...........................[CV] n_estimators=1000, learning_rate=0.05 ...........................



[CV] ... n_estimators=500, learning_rate=0.05, score=0.967758 - 6.2min[C

In [3]:
clf.best_score_ 

0.96771143373321722

In [4]:
clf.best_estimator_

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [7]:
predictions_SVM = list(pred) 
predictions_SVM = zip(range(len(pred)), predictions_SVM)
with open("improved_predictions2430_3.csv","wb") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(["ID", "category"])
    for row in predictions_SVM:
        csv_out.writerow(row)